## Coleta automática - Síntese e Tecnologia

O objetivo desse notebook é automatizar grande parte das coletas do template Síntese e Tecnologia

In [33]:
import os
import time
import urllib3
import pyautogui
import asyncio
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException, ElementNotInteractableException

from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
from urllib.parse import urljoin

Future exception was never retrieved
future: <Future finished exception=Error('Target page, context or browser has been closed')>
playwright._impl._api_types.Error: Target page, context or browser has been closed


As variáveis a seguir guardam informações sobre cada uma das cidades

In [34]:
tags = ["Empenhos","Pagamentos", "folha", "Receitas", "relatorioGF", "relatorioREO",
        "Edital", "Licitacao", "Contrato", "Obra", "orcamento", "RestoAPagar",
        "Contas", "Legislacao", "DiariaDeViagem", 'convenios']
subpastas = ["screenshots", "files", "htmls"]

cidades_urls = {"Janaúba": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PMJanauba", "http://cidadesmg"],
"Caeté": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=CamCaete", "http://cidadesmg"],
"Várzea da Palma": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PMVarzeaDaPalma", "http://cidadesmg"],
"Jaíba": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=JaibaPM", "http://cidadesmg"],
"Capelinha": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PMCapelinha", "http://cidadesmg"],
"Itamarandiba": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Itamarandiba", "http://cidadesmg"],
"Espinosa": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Espnsa", "http://cidadesmg"],
"Minas Novas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=MinasNov", "http://cidadesmg"],
"Novo Cruzeiro": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=pmnovocruzeiro", "http://cidadesmg"],
"Rio Pardo de Minas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PrefRioPMinas", "http://cidadesmg"],
"Buritizeiro": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Buritizeiro", "http://cidadesmg"],
"Coração de Jesus": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=CoracaoDeJes", "http://cidadesmg"],
"Francisco Sá": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=FranciscoSa", "http://cidadesmg"],
"Jequitinhonha": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Jequitinhonha", "http://cidadesmg"],
"Pedra Azul": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PedraAzul", "http://cidadesmg"],
"Serro": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PMSerro", "http://cidadesmg"],
"Monte Azul": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=MonteAzul", "http://cidadesmg"],
"Turmalina": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Turmalina", "http://cidadesmg"],
"Manga": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Manga", "http://cidadesmg"],
"Itacarambi": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PMItacarambi", "http://cidadesmg"],
"Grão Mogol": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=GrMogl", "http://cidadesmg"],
"Mário Campos": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=MarioCampos", "http://cidadesmg"],
"Chapada do Norte": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=chapadanorte", "http://cidadesmg"],
"Itinga": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Itinga", "http://cidadesmg"],
"Virgem da Lapa": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=VirgemLapa", "http://cidadesmg"],
"Água Boa": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=AguaBoa", "http://cidadesmg"],
"Chapada Gaúcha": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=ChapadaGaucha", "http://cidadesmg"],
"Mirabela": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=mirabela", "http://cidadesmg"],
"Águas Vermelhas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PMAguasVermelhas", "http://cidadesmg"],
"São João das Missões": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=SaoJoaoDasMiss", "http://cidadesmg"],
"Mato Verde": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=MatoVerde", "http://cidadesmg"],
"São Romão": ["http://cidadesmg.com.br/portaltransparencia/faces/user/FDiariasViagem.xhtml?Param=SaoRomao", "http://cidadesmg"],
"Setubinha": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Setubinha", "http://cidadesmg"],
"Berilo": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Berilo", "http://cidadesmg"],
"Gouveia": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Gouveia", "http://cidadesmg"],
"Bonito de Minas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=BonitoDeMinas", "http://cidadesmg"],
"Divisópolis": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Divisopolis", "http://cidadesmg"],
"Francisco Badaró": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=FrancBandr", "http://cidadesmg"],
"Lontra": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PmLontra", "http://cidadesmg"],
"Cachoeira de Pajeú": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=CachoeiraPajeu", "http://cidadesmg"],
"Coluna": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Coluna", "http://cidadesmg"],
"Mata Verde": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=MataVerde", "http://cidadesmg"],
"Ibiaí": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PmIbiai", "http://cidadesmg"],
"Montezuma": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Montezuma", "http://cidadesmg"],
"Curral de Dentro": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=CurralDeDentro", "http://cidadesmg"],
"Jenipapo de Minas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=JenipapoMinas", "http://cidadesmg"],
"Claro dos Poções": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=ClarDosPces", "http://cidadesmg"],
"Jequitaí": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Jequitai", "http://cidadesmg"],
"Pintópolis": ["http://cidadesmg.com.br/portaltransparencia/faces/user/FDiariasViagem.xhtml?Param=PMPintopolis", "http://cidadesmg"],
"Nova Porteirinha": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=NovaPorteirinha", "http://cidadesmg"],
"Indaiabira": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=indaiabira", "http://cidadesmg"],
"Santo Antônio do Retiro": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=SantoAntoniodoRetiro", "http://cidadesmg"],
"Engenheiro Navarro": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=EngNavrro", "http://cidadesmg"],
"Divisa Alegre": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=DivisaAlegre", "http://cidadesmg"],
"Mamonas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=mamonas", "http://cidadesmg"],
"Padre Carvalho": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PMPadreCarvalho", "http://cidadesmg"],
"Pai Pedro": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PaiPedro", "http://cidadesmg"],
"Patis": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Patis", "http://cidadesmg"],
"Veredinha": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Veredinha", "http://cidadesmg"],
"Datas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Datas", "http://cidadesmg"],
"Itacambira": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Itacambira", "http://cidadesmg"],
"Fruta de Leite": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=FrutadeLeite", "http://cidadesmg"],
"Novorizonte": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Novorizonte", "http://cidadesmg"],
"Aricanduva": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Aricanduva", "http://cidadesmg"],
"Dores de Guanhães": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=CamDoresDeGanhaes", "http://cidadesmg"],
"Gameleiras": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Gameleiras", "http://cidadesmg"],
"Vargem Grande do Rio Pardo": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=VgmGrnd", "http://cidadesmg"],
"Catuti": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Catuti", "http://cidadesmg"],
"Guaraciama": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PMGuaraciama", "http://cidadesmg"],
"São João da Lagoa": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=SJDLgoa", "http://cidadesmg"],
"Miravânia": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=miravania", "http://cidadesmg"],
"Josenópolis": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Josenopolis", "http://cidadesmg"],
"Felício dos Santos": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=FelicioSantos", "http://cidadesmg"],
"Joaquim Felício": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=JquimFelicio", "http://cidadesmg"],
"Materlândia": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Materlandia", "http://cidadesmg"],
"São João do Pacuí": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=SaoJoaoPacui", "http://cidadesmg"],
"Couto de Magalhães de Minas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=CoutoMagalhaes", "http://cidadesmg"],
"Juramento": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Juramento", "http://cidadesmg"],
"Santana do Riacho": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PmSantanaDoRiacho", "http://cidadesmg"],
"Serra Azul de Minas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=SerraAzulDeMinas", "http://cidadesmg"],
"Senador Modestino Gonçalves": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=SenModestino", "http://cidadesmg"],
"Santa Cruz de Salinas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=SantaCruzDeSalinas", "http://cidadesmg"],
"Alvorada de Minas": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=alvoradademinas", "http://cidadesmg"],
"Frei Lagonegro": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PMFreiLagoNegro", "http://cidadesmg"],
"São Gonçalo do Rio Preto": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=SaoGoncaloDoRioPreto", "http://cidadesmg"],
"Glaucilândia": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=Glaucilandia", "http://cidadesmg"],
"Santo Hipólito": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=SantoHipolito", "http://cidadesmg"],
"Presidente Kubitschek": ["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PresKubitschek", "http://cidadesmg"]}

A seguir, a variável pasta_template guarda o caminho base dos dados

In [35]:
pasta_raiz = os.getcwd()
pasta_template = pasta_raiz + "/Coletas" + "/Sintese/"
print(pasta_template)

/home/arthur/Projeto MP/Coletas/Sintese/


Agora, criam-se as respectivas pastas de subtags para cada uma das cidades

In [36]:
def create_pastes():

    try:
        os.mkdir(pasta_raiz + "/Coletas")
    except FileExistsError:
        pass
    
    try:
        os.mkdir(pasta_raiz + "/Coletas" + "/Sintese/")
    except FileExistsError:
        pass
    
    for cidade in cidades_urls.keys():
        try:
            os.mkdir(pasta_template + cidade)
            for tag in tags:
                os.mkdir(pasta_template  + cidade + "/" + tag)
                for sub in subpastas:
                    os.mkdir(pasta_template  + cidade + "/" + tag + "/" + sub)
        except FileExistsError:
            continue

Aqui encontra-se o crawler que irá separar as páginas do template de acordo com a subtag encontrada na url

In [37]:
def crawl(pages, depth, tags, restriction):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    already_visited = set(pages)
    
    dic = {}
    for tag in tags:
        dic[tag] = set()

    for i in range(depth):
        new_pages = set()
        print("Current depth: " + str(i))

        for page in pages:
            http = urllib3.PoolManager()
            try:
                page_data = http.request('GET', page)
            except:
                print("Error: " + page)
                continue


            soup = BeautifulSoup(page_data.data, "lxml")
            already_visited.add(page)

            links = soup.find_all('a')
            counter = 0

            for link in links:

                if("href" in link.attrs):
                    url = urljoin(page, str(link.get('href')))

                    if url.find("'") != -1:
                        continue

                    url = url.split("#")[0]

                #Restriction to prevent the crawler get out the base url
                if url.startswith(restriction):
                    new_pages.add(url)
                    for tag in tags:
                        if tag in url:
                            dic[tag].add(url)
                                
            counter += 1

        pages = new_pages.difference(already_visited)

        print("Number of links visited: " +  str(len(already_visited)))

    return dic

In [38]:
links = crawl(["http://cidadesmg.com.br/portaltransparencia/faces/user/portal.xhtml?Param=PmIbiai"], 3, tags, "")
print(links)

Current depth: 0


/home/arthur/.local/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


Number of links visited: 1
Current depth: 1
Number of links visited: 15
Current depth: 2
Number of links visited: 52
{'Empenhos': {'http://cidadesmg.com.br/portaltransparencia/faces/user/despesa/FEmpenhosEmitidos.xhtml?Param=PmIbiai', 'http://cidadesmg.com.br/portaltransparencia/faces/user/covid19/FEmpenhosLiquidadosCovid19.xhtml?Param=PmIbiai', 'http://cidadesmg.com.br/portaltransparencia/faces/user/covid19/FEmpenhosAnuladosCovid19.xhtml?Param=PmIbiai', 'http://cidadesmg.com.br/portaltransparencia/faces/user/despesa/FEmpenhosLiquidados.xhtml?Param=PmIbiai', 'http://cidadesmg.com.br/portaltransparencia/faces/user/despesa/FEmpenhosAnulados.xhtml?Param=PmIbiai', 'http://cidadesmg.com.br/portaltransparencia/faces/user/covid19/FEmpenhosEmitidosCovid19.xhtml?Param=PmIbiai'}, 'Pagamentos': {'http://cidadesmg.com.br/portaltransparencia/faces/user/despesa/FPagamentosRealizados.xhtml?Param=PmIbiai', 'http://cidadesmg.com.br/portaltransparencia/faces/user/despesa/FPagamentosTransferencia.xhtml?P

Assim, para cada uma das urls coletadas, aplica-se o coletor base a seguir, que simplesmente abre a página e baixa os dados encontrados como um arquivo CSV

In [39]:
async def coletor1(url, cidade, subtag):

    #try:    
    
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()
    # Open new page
    page = await context.new_page()

    # Go to url
    await page.goto(url)

    # Iterate over years
    for ano in ["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]:
        # Select year
        await page.locator("select[name=\"form\\:j_idt13\"]").select_option(ano)
        await page.locator("button[role=\"button\"]:has-text(\"Consultar\")").click()
        # Click text=CSV
        #try:
        async with page.expect_download() as download_info:
            await page.locator("text=CSV").click()

        download = await download_info.value
        filename = await download.path()
        complement = str(filename).split("/")[-1]

        print(pasta_template + cidade + "/" + subtag + "/files/" +subtag+complement)

        await download.save_as(pasta_template + cidade + "/" + subtag + "/files/" +subtag+complement)

        #except:
         #   print("Erro!!")
          #  break
    # ---------------------
    await context.close()
    await browser.close()
        
    #except:
        #print("Unknown Erro")

Fazendo isso em loop, é possível coletar grande parte dos dados do template rapidamente

In [ ]:
create_pastes()
loop = asyncio.get_event_loop()

for key, value in cidades_urls.items():
    print("Cidade: " + key)
    links_gerados = crawl([value[0]], 3, tags, value[1])
    for tag, links in links_gerados.items():
        print("Coletando " + tag)
        for link in links:
            print(link)
            loop.create_task(coletor1(link, key, tag))
            time.sleep(10)

Comportamento individual funciona corretamente, mas em loop ocorre algum erro!

In [31]:
task = loop.create_task(coletor1("http://cidadesmg.com.br/portaltransparencia/faces/user/despesa/FEmpenhosEmitidos.xhtml?Param=Patis", "Patis", "Empenhos"))

Future exception was never retrieved
future: <Future finished exception=Error('Target page, context or browser has been closed')>
playwright._impl._api_types.Error: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=Error('Target page, context or browser has been closed')>
playwright._impl._api_types.Error: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=Error('Target page, context or browser has been closed')>
playwright._impl._api_types.Error: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=Error('Target page, context or browser has been closed')>
playwright._impl._api_types.Error: Target page, context or browser has been closed
Future exception was never retrieved
future: <Future finished exception=Error('Target page, context or browser has been closed')>
playwright._impl._

/home/arthur/Projeto MP/Coletas/Sintese/Patis/Empenhos/files/Empenhosbeead87d-886b-40f3-a45b-5d54693085a5
/home/arthur/Projeto MP/Coletas/Sintese/Patis/Empenhos/files/Empenhos7eb0459a-3017-415f-ac88-ada17b7a6eb7
/home/arthur/Projeto MP/Coletas/Sintese/Patis/Empenhos/files/Empenhos9225fd2d-a09e-4248-9d6e-2cd8b0566d5d
